<a href="https://colab.research.google.com/github/fahmisyuhada/Klasifikasi-tulisan-tangan-huruf-aksara-sasak-cnn/blob/main/run2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/fahmisyuhada/Klasifikasi-tulisan-tangan-huruf-aksara-sasak-cnn.git projek
import zipfile

data_set = zipfile.ZipFile('projek/character_with_bouding_box.zip', 'r')

data_set.extractall()
data_set.close()
print('dataset unzip successfully')

Cloning into 'projek'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 18 (delta 6), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (18/18), 3.98 MiB | 7.27 MiB/s, done.
Resolving deltas: 100% (6/6), done.
dataset unzip successfully


In [2]:
import numpy as np
np.random.seed

import cv2
import os
from PIL import Image
import keras

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

os.environ['KERAS_BACKEND'] = 'tensorflow'

image_directory = 'karakter3/'
SIZE = 64
dataset = []
label = []

karakter = [['ha',0], ['na',1], ['ca',2], ['ra',3], ['ka',4],
 ['da',5], ['ta',6], ['sa',7], ['wa',8], ['la',9],
  ['ma',10], ['ga',11], ['ba',12], ['nga',13], ['pa',14],
   ['ja',15], ['ya',16], ['nya',17]]

for char in karakter:
  for i, image_name in enumerate(os.listdir(image_directory + char[0] + '/')):
    if(image_name.split('.')[1] == 'jpeg'):
      image = cv2.imread(image_directory + char[0] + '/' + image_name)
      image = Image.fromarray(image, 'RGB')
      image = image.resize((SIZE, SIZE))
      dataset.append(np.array(image))
      label.append(char[1])


In [3]:
X_train, X_test, y_train, y_test = train_test_split(dataset, to_categorical(np.array(label)), test_size = 0.20, random_state = 0)

In [4]:
###########################################
### Second : Using Sequential model #######
#Sequential
#You can create a Sequential model by passing a list of layer instances to the constructor:

from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.models import Sequential

model = None
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape = (SIZE, SIZE, 3), activation = 'relu', data_format='channels_last'))
model.add(MaxPooling2D(pool_size = (2, 2), data_format="channels_last"))
model.add(BatchNormalization(axis = -1))
model.add(Dropout(0.2))
model.add(Convolution2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2), data_format="channels_last"))
model.add(BatchNormalization(axis = -1))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(activation = 'relu', units=512))
model.add(BatchNormalization(axis = -1))
model.add(Dropout(0.2))
model.add(Dense(activation = 'relu', units=256))
model.add(BatchNormalization(axis = -1))
model.add(Dropout(0.2))
model.add(Dense(activation = 'sigmoid', units=5))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 31, 31, 32)        128       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 31, 31, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 32)        0

In [5]:
#import datetime

#log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "/"
#tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


# ### Training the model
# As the training data is now ready, I will use it to train the model.

#Fit the model
history = model.fit(np.array(X_train),
                         y_train,
                         batch_size = 64,
                         verbose = 1,
                         epochs = 25,      #Changed to 3 from 50 for testing purposes.
                         validation_split = 0.1,
                         shuffle = False
                     )

# ## Accuracy calculation
#
# I'll now calculate the accuracy on the test data.


Epoch 1/25
4/4 [==============================] - 8s 894ms/step - loss: 1.5055 - accuracy: 0.4980 - val_loss: 28.7230 - val_accuracy: 0.2500
Epoch 2/25
4/4 [==============================] - 2s 391ms/step - loss: 0.1776 - accuracy: 0.9388 - val_loss: 22.4583 - val_accuracy: 0.2500
Epoch 3/25
4/4 [==============================] - 1s 288ms/step - loss: 0.0676 - accuracy: 0.9959 - val_loss: 16.7415 - val_accuracy: 0.2857
Epoch 4/25
4/4 [==============================] - 1s 327ms/step - loss: 0.0534 - accuracy: 1.0000 - val_loss: 12.4638 - val_accuracy: 0.2857
Epoch 5/25
4/4 [==============================] - 1s 301ms/step - loss: 0.0267 - accuracy: 1.0000 - val_loss: 9.7097 - val_accuracy: 0.3214
Epoch 6/25
4/4 [==============================] - 1s 280ms/step - loss: 0.0192 - accuracy: 1.0000 - val_loss: 7.8214 - val_accuracy: 0.3571
Epoch 7/25
4/4 [==============================] - 1s 314ms/step - loss: 0.0107 - accuracy: 1.0000 - val_loss: 6.6507 - val_accuracy: 0.3214
Epoch 8/25
4/4 [